In [45]:
import pandas as pd
import requests


auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id':     "",
    'client_secret': '',
    'refresh_token': '',
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))
datasets = []
page =1 
page_not_empty = True
while page_not_empty:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': page}
    response = requests.get(activites_url, headers=header, params=param)
    
    if response.status_code != 200:
        print("Failed to retrieve data for page {}".format(page))
        break

    my_dataset = pd.DataFrame(response.json())
    
    if my_dataset.empty:
        break

    datasets.append(my_dataset)
    print("Retrieved page {}".format(page))
    page += 1

    activities_df = pd.concat(datasets, ignore_index=True)
else:
    print("Failed to retrieve access token")






Requesting Token...

Access Token = 92d4e1fe4243163af3698217f6d5522b427bac17

Retrieved page 1


In [46]:
print(activities_df.columns.values)

['resource_state' 'athlete' 'name' 'distance' 'moving_time' 'elapsed_time'
 'total_elevation_gain' 'type' 'sport_type' 'workout_type' 'id'
 'start_date' 'start_date_local' 'timezone' 'utc_offset' 'location_city'
 'location_state' 'location_country' 'achievement_count' 'kudos_count'
 'comment_count' 'athlete_count' 'photo_count' 'map' 'trainer' 'commute'
 'manual' 'private' 'visibility' 'flagged' 'gear_id' 'start_latlng'
 'end_latlng' 'average_speed' 'max_speed' 'has_heartrate'
 'average_heartrate' 'max_heartrate' 'heartrate_opt_out'
 'display_hide_heartrate_option' 'elev_high' 'elev_low' 'upload_id'
 'upload_id_str' 'external_id' 'from_accepted_tag' 'pr_count'
 'total_photo_count' 'has_kudoed']


In [47]:
column_relevant = ['average_heartrate'
,'average_speed'
,'name'
,'distance'
,'max_heartrate'
,'max_speed'
,'moving_time'
,'sport_type'
,'start_date']

activities_df[column_relevant]


,average_heartrate,average_speed,name,distance,max_heartrate,max_speed,moving_time,sport_type,start_date
0,170.2,3.152,Ripetute,5811.7,185.0,4.298,1844,Run,2024-01-22T11:59:33Z
1,157.1,2.511,Lento lungo,18416.5,167.0,3.694,7334,Run,2024-01-20T06:02:38Z
2,156.8,2.490,Lento,10438.5,166.0,3.481,4192,Run,2024-01-19T05:19:28Z
3,172.2,3.150,Ripetute,5402.9,183.0,4.888,1715,Run,2024-01-17T05:16:35Z
4,157.6,2.441,Lento,10581.2,167.0,3.496,4334,Run,2024-01-15T05:12:48Z
...,...,...,...,...,...,...,...,...,...
97,168.8,2.228,Corsa mattutina,5092.5,183.0,4.098,2286,Run,2022-11-18T05:07:26Z
98,170.2,2.412,Corsa pomeridiana,21014.0,182.0,4.094,8712,Run,2022-11-13T15:13:42Z
99,166.7,2.283,Corsa mattutina,7339.1,184.0,3.938,3215,Run,2022-11-07T05:38:44Z
100,161.7,2.146,Corsa dell'ora di pranzo,7692.3,178.0,3.997,3584,Run,2022-11-05T10:38:55Z


In [48]:
activities_df['average_speed_min_km'] =((activities_df['moving_time']/60)/(activities_df['distance']/1000)).astype(int)+(((activities_df['moving_time']/60)/(activities_df['distance']/1000)%1)*0.6)
activities_df['average_speed_km_h'] =((activities_df['distance']/1000)/(activities_df['moving_time']/3600))


In [49]:
# Convertire la colonna 'Data' in un formato di data
activities_df['start_date'] = pd.to_datetime(activities_df['start_date'])

# Ordinare il DataFrame in base alla colonna 'Data'
activities_df = activities_df.sort_values(by='start_date', ascending=True)

# Calcolare la somma cumulativa della colonna 'Valore'
activities_df['cumulative_km'] = (activities_df['distance']/1000).cumsum()

In [50]:
column_relevant = ['average_heartrate'
,'average_speed'
,'average_speed_min_km'
,'average_speed_km_h'
,'name'
,'distance'
,'cumulative_km'
,'max_heartrate'
,'max_speed'
,'moving_time'
,'sport_type'
,'start_date']

activities_df[column_relevant]


,average_heartrate,average_speed,average_speed_min_km,average_speed_km_h,name,distance,cumulative_km,max_heartrate,max_speed,moving_time,sport_type,start_date
101,166.5,2.190,7.366838,7.882916,Corsa pomeridiana,6398.3,6.3983,180.0,4.026,2922,Run,2022-11-04 12:01:22+00:00
100,161.7,2.146,7.459205,7.726641,Corsa dell'ora di pranzo,7692.3,14.0906,178.0,3.997,3584,Run,2022-11-05 10:38:55+00:00
99,166.7,2.283,7.180646,8.217966,Corsa mattutina,7339.1,21.4297,184.0,3.938,3215,Run,2022-11-07 05:38:44+00:00
98,170.2,2.412,6.545808,8.683471,Corsa pomeridiana,21014.0,42.4437,182.0,4.094,8712,Run,2022-11-13 15:13:42+00:00
97,168.8,2.228,7.288954,8.019685,Corsa mattutina,5092.5,47.5362,183.0,4.098,2286,Run,2022-11-18 05:07:26+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
4,157.6,2.441,6.495944,8.789183,Lento,10581.2,1022.3817,167.0,3.496,4334,Run,2024-01-15 05:12:48+00:00
3,172.2,3.150,5.174221,11.341364,Ripetute,5402.9,1027.7846,183.0,4.888,1715,Run,2024-01-17 05:16:35+00:00
2,156.8,2.490,6.415903,8.964361,Lento,10438.5,1038.2231,166.0,3.481,4192,Run,2024-01-19 05:19:28+00:00
1,157.1,2.511,6.382298,9.040005,Lento lungo,18416.5,1056.6396,167.0,3.694,7334,Run,2024-01-20 06:02:38+00:00


In [51]:
activities_df[column_relevant].dtypes

average_heartrate                   float64
average_speed                       float64
average_speed_min_km                float64
average_speed_km_h                  float64
name                                 object
distance                            float64
cumulative_km                       float64
max_heartrate                       float64
max_speed                           float64
moving_time                           int64
sport_type                           object
start_date              datetime64[ns, UTC]
dtype: object

In [52]:
activities_df[column_relevant].to_csv('RunTracker.csv', sep=';')